In [ ]:
# requirement to run this notebook; uncomment install and restart kernel if your environment is missing any of these dependencies
# ! pip install --user --upgrade "transformers>=4.43.2" "peft>=0.7.1,!=0.11.0" "trl>=0.7.9,<0.9.0" bitsandbytes "accelerate>=0.26.1"

In [1]:
import mlflow
import torch
from datasets import load_dataset

from trl import SFTTrainer
from peft import LoraConfig, get_peft_model
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments, AutoTokenizer

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_name = "NousResearch/Meta-Llama-3.1-8B"

use_4_bit = False
use_8_bit = True

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

if use_4_bit:
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,
    )

if use_8_bit:
    bnb_config = BitsAndBytesConfig(load_in_8bit=True)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True,
    torch_dtype=torch.bfloat16 # using an A10G
)
model.config.use_cache = False

dataset_name = "mlabonne/guanaco-llama2-1k"

dataset = load_dataset(dataset_name, split="train")

`low_cpu_mem_usage` was None, now set to True since model is quantized.
Generating train split: 100%|██████████| 1000/1000 [00:00<00:00, 82276.75 examples/s]


In [3]:
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=4,
    logging_dir='./logs',
    logging_steps=10,
    gradient_checkpointing=True,
    report_to=["mlflow"]
)

lora_config = LoraConfig(
        r=8,
        bias="none",
        task_type="CAUSAL_LM",
    )

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    peft_config=lora_config,
    train_dataset=dataset,
    dataset_text_field="text",
)

print("Fine-tuning model:")
with mlflow.start_run() as run:
    trainer.train()

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:246: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
Map: 100%|██████████| 1000/1000 [00:00<00:00, 3539.89 examples/s]


Fine-tuning model:


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:428: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:316: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss
10,1.439700
20,1.375700
30,1.362300
40,1.356400
50,1.438300
60,1.302500
70,1.309700
80,1.376800
90,1.564600
100,1.235800


In [4]:
model_to_save = trainer.model.module if hasattr(trainer.model, 'module') else trainer.model
model_save_location = "/mnt/artifacts/lora/" # save to a Domino dataset for the app and in artifacts for the API
model_to_save.save_pretrained(model_save_location) 

In [6]:
lora_config = LoraConfig.from_pretrained(model_save_location)
model = get_peft_model(model, lora_config)

In [ ]:
text = "Write a concise summary of the key concepts in quantum physics. ### Assistant:"
device = "cuda:0"

inputs = tokenizer(text, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=750)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
